In [3]:
# !wget https://huggingface.co/datasets/mesolitica/malaysian-youtube-audio-instructions/resolve/main/mixtral-audio-instruction.jsonl

In [2]:
import json
import random
from tqdm import tqdm
import re

def clean(string):
    string = re.sub('[^A-Za-z ]+', ' ', string.lower())
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

def overlap(string1, string2):
    l = set([w for w in clean(string1).split() if len(w) > 2])
    r = set([w for w in clean(string2).split() if len(w) > 2])
    return len(l & r) / len(l)

In [4]:
data = []
files = ['mixtral-audio-instruction.jsonl']
for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            data.append(l)

293752it [00:04, 63603.58it/s]


In [ ]:
data = [(i, data[i]) for i in range(len(data))]

In [34]:
ranged = set(range(len(data)))

In [42]:
!rm -rf mining-mixtral-audio-instruction
!mkdir mining-mixtral-audio-instruction

In [43]:
import os

def loop(rows):
    rows, index = rows
    for i in tqdm(range(len(rows))):
        filename = os.path.join('mining-mixtral-audio-instruction', f'{i}-{index}.json')
        if os.path.exists(filename):
            continue
            
        no, d = rows[i]
        
        pairs = []
        list_of_groups = zip(*(iter(d['chat']),) * 2)
        for c in list_of_groups:
            pos = [c[1]['content_ms'], d['context']]
            
            sampled = random.sample(list(ranged - {no}), 100)
            neg = []
            q = c[0]['content']
            for s in sampled:
                try:
                    overlapped = overlap(q, data[s][1]['context'])
                    if overlapped < 0.1:
                        neg.append(data[s][1]['context'])
                    if len(neg) >= 5:
                        break
                except:
                    pass
            
            
            pairs.append(
                {
                    'query': q,
                    'pos': pos,
                    'neg': neg,
                }
            )
        
        with open(filename, 'w') as fopen:
            json.dump(pairs, fopen)

In [44]:
loop((data[:100],0))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 89.58it/s]


In [45]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/2e06e71ef7349a57bc58cc9913ae6bae1f9f8447/mp.py

In [46]:
import mp

mp.multiprocessing(data, loop, cores = 10, returned = False)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29375/29375 [1:00:40<00:00,  8.07it/s]


In [47]:
from glob import glob

In [48]:
files = glob('mining-mixtral-audio-instruction/*.json')
len(files)

293752

In [49]:
with open(files[0]) as fopen:
    d = json.load(fopen)

In [50]:
d

[{'query': 'Why is it more convenient to create our own cable management in a spacious case like the Fractal Design Meshify, which costs around RM439?',
  'pos': ['Apabila ia berkaitan dengan pengurusan kabel, mempunyai sarung yang luas seperti Fractal Design Meshify boleh menjadikan tugas itu lebih mudah dan lebih mudah. Ini disebabkan oleh jumlah ruang yang lebih besar yang tersedia untuk mengatur dan mengalihkan kabel, yang boleh membantu meningkatkan aliran udara, mengurangkan kekusutan kabel dan menjadikannya lebih mudah untuk mengakses komponen untuk penyelenggaraan atau peningkatan.\n\nFractal Design Meshify amat dipandang baik untuk keupayaan pengurusan kabelnya. Sarung ini termasuk beberapa ciri yang direka untuk memudahkan penghalaan kabel, seperti laluan kabel yang digosok getah, pengikat kabel Velcro dan potongan besar di belakang dulang papan induk untuk membolehkan akses mudah ke bahagian belakang papan.\n\nMengenai sebab Fractal Design Meshify dianggap sebagai salah satu

In [51]:
import msgspec

In [52]:
with open('mixtral-audio-instruction-pair.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        with open(f) as fopen:
            d = json.load(fopen)
        for d_ in d:
            if len(d_['neg']) > 3:
                d_['neg'] = random.sample(d_['neg'], 3)
            fopen_l.write(f'{msgspec.json.encode(d_).decode()}\n')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 293752/293752 [00:32<00:00, 8991.57it/s]


In [53]:
!wc -l mixtral-audio-instruction-pair.jsonl

741980 mixtral-audio-instruction-pair.jsonl


In [57]:
!ls -lh mixtral-audio-instruction-pair.jsonl

-rw-r--r-- 1 husein husein 2.4G Mac   6 19:17 mixtral-audio-instruction-pair.jsonl


In [58]:
!head -n 1 mixtral-audio-instruction-pair.jsonl

{"query":"Why is it more convenient to create our own cable management in a spacious case like the Fractal Design Meshify, which costs around RM439?","pos":["Apabila ia berkaitan dengan pengurusan kabel, mempunyai sarung yang luas seperti Fractal Design Meshify boleh menjadikan tugas itu lebih mudah dan lebih mudah. Ini disebabkan oleh jumlah ruang yang lebih besar yang tersedia untuk mengatur dan mengalihkan kabel, yang boleh membantu meningkatkan aliran udara, mengurangkan kekusutan kabel dan menjadikannya lebih mudah untuk mengakses komponen untuk penyelenggaraan atau peningkatan.\n\nFractal Design Meshify amat dipandang baik untuk keupayaan pengurusan kabelnya. Sarung ini termasuk beberapa ciri yang direka untuk memudahkan penghalaan kabel, seperti laluan kabel yang digosok getah, pengikat kabel Velcro dan potongan besar di belakang dulang papan induk untuk membolehkan akses mudah ke bahagian belakang papan.\n\nMengenai sebab Fractal Design Meshify dianggap sebagai salah satu kes t

In [60]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='mixtral-audio-instruction-pair.jsonl',
    path_in_repo='mixtral-audio-instruction-pair.jsonl',
    repo_id='mesolitica/instructions-pair-mining', 
    repo_type='dataset',
)

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/instructions-pair-mining/commit/4dec1250e4bec2ac0cb7b89bde4591878f9b752a', commit_message='Upload mixtral-audio-instruction-pair.jsonl with huggingface_hub', commit_description='', oid='4dec1250e4bec2ac0cb7b89bde4591878f9b752a', pr_url=None, pr_revision=None, pr_num=None)